In [77]:
import gensim
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Flatten, Input

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\mdgha\OneDrive\Documents\Ghayaz Personal\Data Science and Analytics\Emoji Prediction\Train.csv")
df

,Unnamed: 0,TEXT,Label
0,0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,2,Been friends since 7th grade. Look at us now w...,2
3,3,This is what it looks like when someone loves ...,3
4,4,RT @user this white family was invited to a Bl...,3
...,...,...,...
69995,69995,"Yes, I call Galina ""my Bubie"" Go follow my bea...",3
69996,69996,"I SEA you, Seattle @ Ballard Seafood Festival\n",16
69997,69997,If one of my daughters is wearing this and ask...,2
69998,69998,Guess who whoop people on THEIR homecoming?! #...,3


In [4]:
df.drop(columns='Unnamed: 0', inplace=True)
df

,TEXT,Label
0,Vacation wasted ! #vacation2017 #photobomb #ti...,0
1,"Oh Wynwood, you’re so funny! : @user #Wynwood ...",1
2,Been friends since 7th grade. Look at us now w...,2
3,This is what it looks like when someone loves ...,3
4,RT @user this white family was invited to a Bl...,3
...,...,...
69995,"Yes, I call Galina ""my Bubie"" Go follow my bea...",3
69996,"I SEA you, Seattle @ Ballard Seafood Festival\n",16
69997,If one of my daughters is wearing this and ask...,2
69998,Guess who whoop people on THEIR homecoming?! #...,3


In [7]:
emoji_df = pd.read_csv(r"C:\Users\mdgha\OneDrive\Documents\Ghayaz Personal\Data Science and Analytics\Emoji Prediction\Mapping.csv")
emoji_df.drop(columns='Unnamed: 0', inplace=True)
emojis = np.array(emoji_df['emoticons'])
emojis

array(['😜', '📸', '😍', '😂', '😉', '🎄', '📷', '🔥', '😘', '❤', '😁', '🇺🇸', '☀',
       '✨', '💙', '💕', '😎', '😊', '💜', '💯'], dtype=object)

In [9]:
tokenized_sentence = df.TEXT.apply(gensim.utils.simple_preprocess)

In [10]:
tokenized_sentence

0        [vacation, wasted, vacation, photobomb, tired,...
1        [oh, wynwood, you, re, so, funny, user, wynwoo...
2        [been, friends, since, th, grade, look, at, us...
3        [this, is, what, it, looks, like, when, someon...
4        [rt, user, this, white, family, was, invited, ...
                               ...                        
69995    [yes, call, galina, my, bubie, go, follow, my,...
69996      [sea, you, seattle, ballard, seafood, festival]
69997    [if, one, of, my, daughters, is, wearing, this...
69998    [guess, who, whoop, people, on, their, homecom...
69999    [we, love, you, robbie, heritage, memorial, ce...
Name: TEXT, Length: 70000, dtype: object

In [11]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2,
    workers = 4
)

In [12]:
model.build_vocab(tokenized_sentence, progress_per=1000)

In [13]:
model.train(tokenized_sentence, total_examples=model.corpus_count, epochs= model.epochs)

(2787873, 3583330)

In [20]:
model.wv.most_similar("yes")

[('someone', 0.949273943901062),
 ('cause', 0.941557765007019),
 ('actually', 0.9322378635406494),
 ('guess', 0.926924467086792),
 ('expect', 0.9265212416648865),
 ('said', 0.9207907319068909),
 ('really', 0.9196456074714661),
 ('says', 0.919119656085968),
 ('knows', 0.9143480658531189),
 ('grow', 0.9140030145645142)]

In [21]:
def get_word2vec_embeddings(tokenized_sentence, word2vec_model):
    embedding_matrix = []
    for word in tokenized_sentence:
        if word in model.wv:
            embedding_matrix.append(model.wv[word])
        else:
            embedding_matrix.append(np.zeros(word2vec_model.vector_size))
    return np.array(embedding_matrix)

In [36]:
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

maxlen = 15


In [61]:
embedded_matrix = []
maxlen = 15

for sentence in tokenized_sentence:
    if len(sentence) >= maxlen:
        embedded_matrix.append(get_word2vec_embeddings(sentence[:maxlen], model))
    elif len(sentence) < maxlen:
        sentence = list(sentence)
        sentence.extend(np.zeros(maxlen - len(sentence)))
        embedded_matrix.append(get_word2vec_embeddings(sentence, model))

In [62]:
list(embedded_matrix)

[array([[-0.23774415,  0.38572121, -0.24521302, ..., -0.62840021,
          0.04621602, -0.06350954],
        [ 0.00708011,  0.04305067, -0.02663384, ..., -0.01987696,
         -0.01309963, -0.0265223 ],
        [-0.23774415,  0.38572121, -0.24521302, ..., -0.62840021,
          0.04621602, -0.06350954],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]),
 array([[ 0.35642406,  0.57489812, -0.19127876, ..., -0.28276482,
          0.58269703,  0.03967984],
        [-0.09238385,  0.40344939,  0.08080383, ...,  0.08317913,
         -0.05014766,  0.17278573],
        [ 0.69330847,  1.19471169, -1.81465471, ...,  0.02465102,
          0.46437559,  0.58208942],
        ...,
        [-0.34972849,  0.42574146, -0.0607224 , ..., -

In [123]:
embedded_matrix.shape

(70000, 15, 100)

In [53]:
Y_train = to_categorical(df.Label)
Y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [54]:
from tensorflow.keras.models import Sequential

In [57]:
model.vector_size

100

In [81]:
embedded_matrix = np.array(embedded_matrix)
embedded_matrix.shape

(70000, 15, 100)

In [90]:
network = Sequential([
    Input(shape=(15, 100)),
    LSTM(128, return_sequences=True),
    LSTM(16, return_sequences=False),
    Flatten(),
    Dense(16, activation='relu'),
    Dense(20, activation='softmax')
])

network.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [79]:
Y_train.shape

(70000, 20)

In [93]:
network.fit(embedded_matrix, Y_train, epochs=100)

Epoch 1/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 59s 27ms/step - accuracy: 0.3649 - loss: 2.1206
Epoch 2/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 56s 25ms/step - accuracy: 0.3721 - loss: 2.0817
Epoch 3/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 55s 25ms/step - accuracy: 0.3782 - loss: 2.0530
Epoch 4/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 60s 27ms/step - accuracy: 0.3915 - loss: 2.0093
Epoch 5/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 55s 25ms/step - accuracy: 0.3996 - loss: 1.9829
Epoch 6/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 57s 26ms/step - accuracy: 0.4045 - loss: 1.9559
Epoch 7/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 56s 25ms/step - accuracy: 0.4156 - loss: 1.9091
Epoch 8/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 54s 25ms/step - accuracy: 0.4237 - loss: 1.8760
Epoch 9/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 53s 24ms/step - accuracy: 0.4357 - loss: 1.8378
Epoch 10/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 54s 24ms/step - accuracy: 0.4407 - loss: 1.8137
Epoch 11/100
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 55s 25ms/step - accuracy: 0.4458 - loss: 1.79

In [183]:
test = "Fire man"
test_token = gensim.utils.simple_preprocess(test)

# Create a Word2Vec model with vector size 100
test_model = gensim.models.Word2Vec([test_token], vector_size=100, window=5, min_count=1, workers=4)

# Define maximum length
maxlen = 15

# Function to convert tokens to embeddings
def get_word2vec_embeddings(tokens, model):
    embeddings = []
    for token in tokens:
        if token in model.wv:
            embeddings.append(model.wv[token])
        else:
            embeddings.append(np.zeros(model.vector_size))  # Pad unknown words with zeros
    return np.array(embeddings)

# Pad or truncate the token list to 15
test_token = list(test_token)
if len(test_token) < maxlen:
    test_token.extend(['<PAD>'] * (maxlen - len(test_token)))  # Use <PAD> for padding
else:
    test_token = test_token[:maxlen]  # Truncate if too long

# Convert tokens to embeddings (using padding for <PAD>)
X_test = get_word2vec_embeddings(test_token, model)

# Add padding vectors for <PAD>
X_test = np.array([test_model.wv[token] if token in test_model.wv else np.zeros(100) for token in test_token])

print("X_test shape:", X_test.shape)  # Should be (15, 100)


X_test shape: (15, 100)


In [185]:
X_test

array([[-0.00861969,  0.00366574,  0.00518988, ..., -0.00239151,
        -0.00951009,  0.00450588],
       [-0.00053623,  0.00023643,  0.00510335, ..., -0.00704156,
         0.00090146,  0.00639253],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [179]:
X_test.shape

(15, 100)

In [180]:
from tensorflow import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.TEXT.values)

In [181]:
X_test = np.expand_dims(X_test, axis=0)  # Adds a batch dimension

# Now X_test shape will be (1, 15, 100)


y_pred = network.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


In [182]:
y_pred

array([2], dtype=int64)